<a href="https://colab.research.google.com/github/daisuke-8080/blood_tree.xlsx_maker/blob/main/blood_tree_xlsx_(%E4%B9%B3%E7%89%9B%E3%81%AE%E8%A1%80%E7%B5%B1%E6%A8%B9%E3%81%AE%E3%82%A8%E3%82%AF%E3%82%BB%E3%83%AB%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB)%E3%81%AE%E4%BD%9C%E6%88%90%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***下のボックスを実行した後、ランタイムを再起動してください***

In [ ]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 6.0 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


### **下のボックスの最初の緑の文字の指示に従ってから実行ボタンを押してください。**
### **そのあとはすぐにcode_to_excel関数やall_to_excel関数を実行できるように変更しました。**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import openpyxl

df = pd.DataFrame()

#下のrange()の中に「xlsxファイルの数」-1 を代入する
for i in range(10):
  i += 1
  df_i = pd.read_excel(f'/content/drive/MyDrive/乳牛データ/{i}.xlsx',sheet_name='Holstein - Results', header=1, index_col=None)
  df = pd.concat((df, df_i), axis=0)

df.reset_index(drop=True, inplace=True)

#「母」列のコードを下１０桁に揃える
for i in range(len(df)):
  df.loc[i, ['母']] = df.loc[i, ['母']].item()[-10:]
  df.loc[i , ['牧場ID (牧場管理番号)']] = df.loc[i , ['牧場ID (牧場管理番号)']].astype(str)
  if len(df.loc[i, ['牧場ID (牧場管理番号)']].item()) !=10:
    raise

#「娘」列を３列追加
df[['娘1', '娘2', '娘3']] = np.nan

#全ての行における「母」と「牧場ID (牧場管理番号)」をリストとしてmom_codes, idsに格納
mom_codes = df[['母']].values.squeeze(1).tolist()
ids = df[['牧場ID (牧場管理番号)']].values.squeeze(1).tolist()


#ターゲットとなるi行の「ID」が他のj行の「母」と一致していたら、そのターゲットi行の「娘」列にj行のIDを代入する（娘1 - 娘3まで）
daughter_num = 1
for i, id in enumerate(ids):
  for j, mom in enumerate(mom_codes):
    if id == mom:
      df.loc[ i,[f'娘{daughter_num}']] = df.loc[ j, ['牧場ID (牧場管理番号)']].values
      daughter_num += 1
  daughter_num = 1


#ターゲットとなるi行の「母」が ids (全てのIDのリスト)　に含まれていなかったらそのi行目の id を第一世代 first_generationsとし、
#その逆に、含まれていなかったら第二世代以降 offspringであると判定
first_generations = []
offspring = []

for i in range(len(ids)):
  id = ids[i]
  if df.loc[ i, ['母']].values.item() not in ids:
    first_generations.append(id)
  else:
    offspring.append(id)
#重複除去
first_generations = list(dict.fromkeys(first_generations))
offspring = list(dict.fromkeys(offspring))


from operator import itemgetter
nan_type_py = type(float('nan'))
nan_type_np = type(np.float64(1))
main_list = []
for i in range(len(df)):
  partial_tuple = df.loc[ i, ['牧場ID (牧場管理番号)', '娘1','娘2', '娘3']].values.tolist()
  #partial_tuple はリストだがすぐにタプル化される。　ex ['1386116606', '1439421916', 'nan', 'nan']
  for daughter_num in range(3):
    daughter_num += 1
    partial_list = list(itemgetter(0, daughter_num)(partial_tuple))
    #partial_listは partial_tuple のターゲットi行の ID とその娘のIDの2要素からなる部分リスト ex ['1386116606', '1439421916']
    #もしこのpartial_listの二つ目の要素が nan だった場合、あとあと扱いやすいよう nan のタイプをfloat などから文字列 'nan'　に揃える
    if (type(partial_list[1]) == nan_type_py) | (type(partial_list[1]) == nan_type_np):
      partial_list[1] = 'nan'
    #main_listにすべてのpartial_listを格納していく ex [['1386116606', '1439421916'], ['1386116606', 'nan'], ['1386116606', 'nan'], ・・・・・]
    main_list.append(partial_list)

#main_list_2 はmain_listの 'nan' による重複を除去したリスト。　各要素は ID とその娘のIDの2要素からなるリスト
main_list_2 = []
for partial in main_list:
  if partial not in main_list_2:
    main_list_2.append(partial)


#main_list_2の中のリスト同士の連結を行うためのwhileループ
loop_flag = True
previous_max_length = 2
while loop_flag:
  
  for target in main_list_2:
    target_removal_flag = False

    for partial in main_list_2:
      # ターゲットとなるリスト targetの先頭の要素が、他のリスト partialのしっぽの要素と一致していたらtargetのコピーとpartialを連結する
      # 例えば targetが ['1386116606', '1439421916']、 partialが　['1439421916', 'nan']の時、 target_copyは['1386116606', '1439421916'', '1439421916', 'nan']となる
      if target[-1] == partial[0]:
        target_removal_flag = True
        target_copy = target.copy()
        target_copy.extend(partial)
        new_partial = list(dict.fromkeys(target_copy))
        # new_partialはtarget_copyの重複を除去したリスト。この new_partialの長さが最長記録previous_max_lengthを超えたら、new_max_lengthにこの長さを登録
        if len(new_partial) > previous_max_length:
          new_max_length = len(new_partial)
        main_list_2.append(new_partial)
        main_list_2.remove(partial)

    #元のtarget は除去する
    if target_removal_flag:
      main_list_2.remove(target)

  # main_list_2の中のすべてのリストを target にし終えて、new_max_lengthがprevious_max_lengthと同じだった場合、main_list_2の中のリスト同士をこれ以上連結できないとみなし、whileループを抜ける
  if previous_max_length == new_max_length:
    loop_flag = False
  else:
    previous_max_length = new_max_length

#同一の第一世代を持つ横ベクトルを並べたDataFrameをもつ辞書（keyはそれらの第一世代のID）
df_dict = {}
for first_gen_id in first_generations:
  # df_xは　df_dictに入れる用のDataFrameの土台
  df_x = pd.DataFrame()
  # 第一世代のリストfirst_generationsの各要素 first_gen_idに注目する。main_list_2の中のリスト partialの先頭のIDが第一世代のID first_gen_idと一致していたらリスト partial を新たな リスト main_partial_listに格納
  main_partial_list = [partial for partial in main_list_2 if partial[0] == first_gen_id]

  #main_partial_listの中のリスト partial が2つ以上存在するとき、
  if len(main_partial_list) > 1:
    for partial in main_partial_list:
      #もし partial が first_gen_id と 'nan'の2要素しか持たない場合、df_xに加える作業を無視する
      if len(partial) == 2:
        continue

      #もし partial が first_gen_id と 'nan'以外の要素をもつリストである場合、
      else:
        for i, id in enumerate(partial):
          if id == 'nan':
            continue
          #partialの 'nan' でない id を、大元のdf からDataFrame（横ベクトル）として 抽出し、コピーする。インデックスを世代番号に変更して df_x に追加
          df_row = df[(df['牧場ID (牧場管理番号)']==id)].copy()
          index = df_row.index.item()
          df_row.rename(index={index : i+1}, inplace=True)
          df_x = df_x.append(df_row)
        # partial ごとに空行を挟む
        df_x = df_x.append(pd.Series(name=''))
  
  
  #main_partial_listの中のリスト partial が一つしか存在しいないとき、
  #partialの 'nan' でない id を、大元のdf からDataFrame（横ベクトル）として 抽出し、コピー。インデックスを世代番号に変更して df_x に追加
  else:
    id = main_partial_list[0][0]
    df_row = df[(df['牧場ID (牧場管理番号)']==id)].copy()
    index = df_row.index.item()
    df_row.rename(index={index : 1}, inplace=True)
    df_x = df_x.append(df_row)
    df_x = df_x.append(pd.Series(name=''))

  # 最終的に key を first_gen_id として、df_x を df_dictに格納
  df_dict[first_gen_id] = df_x
  

# code_confering_dictは、全てのIDを key として持ち、それらの key の「第一世代」のID を value として持つ辞書
# この辞書を使って任意のIDの第一世代を参照する
code_confering_dict = {}
for id in ids:
  # ターゲットとなる id がfirst_genterationsに含まれていたらそれをvalueに登録
  if id in first_generations:
    code_confering_dict[id] = id

  # ターゲットとなる id がfirst_genterationsに含まれていない場合、
  else:
    #そのidの母のidをmom_codeとし、mom_codeがfirst_generationsに含まれるまでmom_codeを更新し続ける。
    mom_code = df[(df['牧場ID (牧場管理番号)'] == id)]['母'].values.item()
    while mom_code not in first_generations:
      mom_code = df[(df['牧場ID (牧場管理番号)'] == mom_code)]['母'].values.item()
    #最終的にfirst_generationsに含まれるmom_codeを value として登録する
    code_confering_dict[id] = mom_code





#選択したIDの第一世代のDataFrameからxlsxファイルを作成する関数
def code_to_excel(code):
  #特徴量選択
  main_specifics = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母', '生年月日', 'DWP$', 'DWP$ % Rank★90％＝上位10％',  '乳量(ポンド）', '体細胞\nスコア', '娘牛\n妊娠率', '生産寿命', 'Zoetis\u3000\u3000\u3000\u3000乳房炎',  'Zoetis\u3000\u3000\u3000\u3000跛行\u3000', 'Zoetis\u3000\u3000\u3000\u3000子宮炎',  '乳牛健康形質', 'Zoetis\u3000子牛生存能力', '乳器' ,'高さ', '近交係数']
  only_blood = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母']
  code = str(code)
  #code_confering_dict を使って選択したIDの第一世代のコードを取得する
  code = code_confering_dict[code]
  df_target = df_dict[code]

  path = '/content/drive/MyDrive/乳牛データ/id.xlsx'
  path = path.replace('id', code)

  with pd.ExcelWriter(path) as writer:
    df_target[only_blood].to_excel(writer, sheet_name='系統樹(小)')
    df_target[main_specifics].to_excel(writer, sheet_name='系統樹(メイン)')
    df_target.to_excel(writer, sheet_name='系統樹(大)')
    
  return




#全ての第一世代のDataFrameからxlsxファイルを作成する関数
def all_to_excel():
  #特徴量選択
  main_specifics = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母', '生年月日', 'DWP$', 'DWP$ % Rank★90％＝上位10％',  '乳量(ポンド）', '体細胞\nスコア', '娘牛\n妊娠率', '生産寿命', 'Zoetis\u3000\u3000\u3000\u3000乳房炎',  'Zoetis\u3000\u3000\u3000\u3000跛行\u3000', 'Zoetis\u3000\u3000\u3000\u3000子宮炎',  '乳牛健康形質', 'Zoetis\u3000子牛生存能力', '乳器' ,'高さ', '近交係数']
  only_blood = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母']
  
  # xlsxの土台となるDataFrame
  df_all = pd.DataFrame()
  
  #df_dictに格納されている全ての第一世代のdfを連結してdf_allを作成
  for code in first_generations:
    df_target = df_dict[code]
    df_all = pd.concat((df_all, df_target), axis=0)

  path = '/content/drive/MyDrive/乳牛データ/all_blood_tree.xlsx'

  with pd.ExcelWriter(path) as writer:
    df_all[only_blood].to_excel(writer, sheet_name='系統樹(小)')
    df_all[main_specifics].to_excel(writer, sheet_name='系統樹(メイン)')
    df_all.to_excel(writer, sheet_name='系統樹(大)')
    
  return




#選択したIDの第一世代のDataFrameからxlsxファイルを作成する関数  (重複なしバージョン)
def code_to_excel_without_duplication(code):
  #特徴量選択
  main_specifics = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母', '生年月日', 'DWP$', 'DWP$ % Rank★90％＝上位10％',  '乳量(ポンド）', '体細胞\nスコア', '娘牛\n妊娠率', '生産寿命', 'Zoetis\u3000\u3000\u3000\u3000乳房炎',  'Zoetis\u3000\u3000\u3000\u3000跛行\u3000', 'Zoetis\u3000\u3000\u3000\u3000子宮炎',  '乳牛健康形質', 'Zoetis\u3000子牛生存能力', '乳器' ,'高さ', '近交係数']
  only_blood = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母']
  code = str(code)
  #code_confering_dict を使って選択したIDの第一世代のコードを取得する
  code = code_confering_dict[code]
  df_target = df_dict[code]
      #重複の削除等
  target_index = df_target.loc[['']].index
  df_target = df_target.drop(target_index).sort_index().drop_duplicates()
  df_target = df_target.append(pd.Series(name=''))

  path = '/content/drive/MyDrive/乳牛データ/id.xlsx'
  path = path.replace('id', code + "_without_duplication")

  with pd.ExcelWriter(path) as writer:
    df_target[only_blood].to_excel(writer, sheet_name='系統樹(小)')
    df_target[main_specifics].to_excel(writer, sheet_name='系統樹(メイン)')
    df_target.to_excel(writer, sheet_name='系統樹(大)')
    
  return




#全ての第一世代のDataFrameからxlsxファイルを作成する関数　(重複なしバージョン)
def all_to_excel_without_duplication():
  #特徴量選択
  main_specifics = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母', '生年月日', 'DWP$', 'DWP$ % Rank★90％＝上位10％',  '乳量(ポンド）', '体細胞\nスコア', '娘牛\n妊娠率', '生産寿命', 'Zoetis\u3000\u3000\u3000\u3000乳房炎',  'Zoetis\u3000\u3000\u3000\u3000跛行\u3000', 'Zoetis\u3000\u3000\u3000\u3000子宮炎',  '乳牛健康形質', 'Zoetis\u3000子牛生存能力', '乳器' ,'高さ', '近交係数']
  only_blood = ['検査日', '牧場ID (牧場管理番号)', '品種', '父\nNAABコード', '母']

  # xlsxの土台となるDataFrame
  df_all = pd.DataFrame()
  
  #df_dictに格納されている全ての第一世代のdfを連結してdf_allを作成
  for code in first_generations:
    df_target = df_dict[code]
      #重複の削除等
    target_index = df_target.loc[['']].index
    df_target = df_target.drop(target_index).sort_index().drop_duplicates()
    df_target = df_target.append(pd.Series(name=''))
    df_all = pd.concat((df_all, df_target), axis=0)

  path = '/content/drive/MyDrive/乳牛データ/all_blood_tree_without_duplication.xlsx'

  with pd.ExcelWriter(path) as writer:
    df_all[only_blood].to_excel(writer, sheet_name='系統樹(小)')
    df_all[main_specifics].to_excel(writer, sheet_name='系統樹(メイン)')
    df_all.to_excel(writer, sheet_name='系統樹(大)')
    
  return

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:152: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:141: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
#大元のDataFrame
df

In [ ]:
#大元のDataFrameの基本情報
df.info()

In [ ]:
#第一世代の全てのIDのリスト
first_generations

In [ ]:
#第二世代以降の全てのIDのリスト
offspring

In [ ]:
#少なくとも一匹の娘を持つ牛のDataFrame
df.dropna(subset=['娘1'])

In [ ]:
# 血統樹の樹形図を一本ずつベクトルにバラしたものを集めたリスト（ベクトルのリスト）
main_list_2

In [ ]:
# main_list_２の要素数　＝　全ての樹形図における「道」の本数の合計
len(main_list_2)

In [ ]:
#　'1386116606'　の位置に任意の第一世代のIDを代入して実行すると樹形図をベクトルにばらしたものが表示される
main_partial_list = [partial for partial in main_list_2 if partial[0] == '1386116606']
main_partial_list

In [ ]:
#　'1386116606'　の位置に任意のIDを代入して実行するとそのIDにとっての第一世代の牛のIDが表示される
code_confering_dict['1386116606']

### **下の四つの関数をお好みで実行してください。IDの参照には上のコードを適宜利用してください**

*   code_to_excel()
 選択したIDの第一世代のDataFrameからxlsxファイルを作成する関数
*   
code_to_excel()選択したIDの第一世代のDataFrameからxlsxファイルを作成する関数 (重複なしバージョン)
*   all_to_excel_without_dupliication() 全ての第一世代のDataFrameからxlsxファイルを作成する関数 
*   all_to_excel_without_dupliication() 全ての第一世代のDataFrameからxlsxファイルを作成する関数  (重複なしバージョン)



In [ ]:
#選択したIDの第一世代のDataFrameからxlsxファイルを作成する関数。　1386116606の位置に任意のIDを代入してください
code_to_excel(1386116606)

In [ ]:
#選択したIDの第一世代のDataFrameからxlsxファイルを作成する関数  (重複なしバージョン)。　1386116606の位置に任意のIDを代入してください
code_to_excel_without_duplication(1386116606)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:241: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
#全ての第一世代のDataFrameからxlsxファイルを作成する関数
all_to_excel()

In [ ]:
#全ての第一世代のDataFrameからxlsxファイルを作成する関数  (重複なしバージョン)
all_to_excel_without_duplication()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:271: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
